In [71]:
import pandas as pd
import requests
import traceback
from bs4 import BeautifulSoup
%matplotlib inline

#Flat File

In [115]:
csv_movies_with_budget_df=pd.read_csv(r'./project_datasets/top-500-movies.csv', parse_dates=True)
csv_movies_with_budget_df.head()

,rank,release_date,title,url,production_cost,domestic_gross,worldwide_gross,opening_weekend,mpaa,genre,theaters,runtime,year
0,1,2019-04-23,Avengers: Endgame,/movie/Avengers-Endgame-(2019)#tab=summary,400000000,858373000,2797800564,357115007.0,PG-13,Action,4662.0,181.0,2019.0
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,/movie/Pirates-of-the-Caribbean-On-Stranger-Ti...,379000000,241071802,1045713802,90151958.0,PG-13,Adventure,4164.0,136.0,2011.0
2,3,2015-04-22,Avengers: Age of Ultron,/movie/Avengers-Age-of-Ultron#tab=summary,365000000,459005868,1395316979,191271109.0,PG-13,Action,4276.0,141.0,2015.0
3,4,2015-12-16,Star Wars Ep. VII: The Force Awakens,/movie/Star-Wars-Ep-VII-The-Force-Awakens#tab=...,306000000,936662225,2064615817,247966675.0,PG-13,Adventure,4134.0,136.0,2015.0
4,5,2018-04-25,Avengers: Infinity War,/movie/Avengers-Infinity-War#tab=summary,300000000,678815482,2048359754,257698183.0,PG-13,Action,4474.0,156.0,2018.0


In [136]:
cst_min,cst_max = csv_movies_with_budget_df["production_cost"].agg(['min', 'max'])
step=5
div=1_000_000
cost_range=[(start-step,start) for start in range(int(cst_min/div),int(cst_max/div)+step,step)]

In [137]:
# Define "nice-looking" labels (otherwise x axis will read "(4.9, 9.1] (9.9, 14.1] .....")
labels=[f"{start}-{end}" for start, end in cost_range]
# Define the bins
bins=pd.IntervalIndex.from_tuples(cost_range)
bins

IntervalIndex([  (86, 91],   (91, 96],  (96, 101], (101, 106], (106, 111],
               (111, 116], (116, 121], (121, 126], (126, 131], (131, 136],
               (136, 141], (141, 146], (146, 151], (151, 156], (156, 161],
               (161, 166], (166, 171], (171, 176], (176, 181], (181, 186],
               (186, 191], (191, 196], (196, 201], (201, 206], (206, 211],
               (211, 216], (216, 221], (221, 226], (226, 231], (231, 236],
               (236, 241], (241, 246], (246, 251], (251, 256], (256, 261],
               (261, 266], (266, 271], (271, 276], (276, 281], (281, 286],
               (286, 291], (291, 296], (296, 301], (301, 306], (306, 311],
               (311, 316], (316, 321], (321, 326], (326, 331], (331, 336],
               (336, 341], (341, 346], (346, 351], (351, 356], (356, 361],
               (361, 366], (366, 371], (371, 376], (376, 381], (381, 386],
               (386, 391], (391, 396], (396, 401]],
              dtype='interval[int64, right]')

In [138]:
# Use "cut" method to group values into ranges
csv_movies_with_budget_df['prod_cost_range_million'] = pd.cut(csv_movies_with_budget_df['production_cost'].apply(lambda x : int(x/div)),
                                                              bins=bins, include_lowest=True
                                                              ).map(dict(zip(bins, labels)))

In [139]:
#Ensure all rows are accounted for
csv_movies_with_budget_df[csv_movies_with_budget_df["prod_cost_range_million"].isna()]

,rank,release_date,title,url,production_cost,domestic_gross,worldwide_gross,opening_weekend,mpaa,genre,theaters,runtime,year,prod_cost_range_million


In [140]:
# Count the occurrences of each  range
csv_movies_with_budget_df.prod_cost_range_million.value_counts()

prod_cost_range_million
96-101     59
146-151    50
196-201    40
106-111    33
91-96      25
           ..
306-311     0
251-256     0
266-271     0
291-296     0
241-246     0
Name: count, Length: 63, dtype: int64

In [135]:
csv_movies_with_budget_df

,rank,release_date,title,url,production_cost,domestic_gross,worldwide_gross,opening_weekend,mpaa,genre,theaters,runtime,year,prod_cost_range_million
0,1,2019-04-23,Avengers: Endgame,/movie/Avengers-Endgame-(2019)#tab=summary,400000000,858373000,2797800564,357115007.0,PG-13,Action,4662.0,181.0,2019.0,396-401
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,/movie/Pirates-of-the-Caribbean-On-Stranger-Ti...,379000000,241071802,1045713802,90151958.0,PG-13,Adventure,4164.0,136.0,2011.0,376-381
2,3,2015-04-22,Avengers: Age of Ultron,/movie/Avengers-Age-of-Ultron#tab=summary,365000000,459005868,1395316979,191271109.0,PG-13,Action,4276.0,141.0,2015.0,361-366
3,4,2015-12-16,Star Wars Ep. VII: The Force Awakens,/movie/Star-Wars-Ep-VII-The-Force-Awakens#tab=...,306000000,936662225,2064615817,247966675.0,PG-13,Adventure,4134.0,136.0,2015.0,301-306
4,5,2018-04-25,Avengers: Infinity War,/movie/Avengers-Infinity-War#tab=summary,300000000,678815482,2048359754,257698183.0,PG-13,Action,4474.0,156.0,2018.0,296-301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,2013-02-06,A Good Day to Die Hard,/movie/Die-Hard-5#tab=summary,92000000,67349198,304249198,24834845.0,R,Action,3555.0,98.0,2013.0,91-96
496,497,2004-04-09,The Alamo,/movie/Alamo-The-(2004)#tab=summary,92000000,22406362,23911362,9124701.0,PG-13,Western,2609.0,137.0,2004.0,91-96
497,498,1995-12-22,Cutthroat Island,/movie/Cutthroat-Island#tab=summary,92000000,10017322,18517322,2371415.0,PG-13,Adventure,1619.0,NaN,1995.0,91-96
498,499,2013-12-19,The Secret Life of Walter Mitty,/movie/Secret-Life-of-Walter-Mitty-The-(2012)#...,91000000,58236838,187861183,12765508.0,PG,Adventure,2922.0,114.0,2013.0,86-91


In [79]:
csv_movies_with_budget_df.columns

Index(['rank', 'release_date', 'title', 'url', 'production_cost',
       'domestic_gross', 'worldwide_gross', 'opening_weekend', 'mpaa', 'genre',
       'theaters', 'runtime', 'year'],
      dtype='object')

In [66]:
csv_movies_with_budget_df.shape

(500, 13)

#### WEB Scraping


>Speaking about WebScraping the box office hits, the most complicated part of this is inspecting the webpage source code to determine what to grab and what to ignore.
I started off to create a dataset from the weekend performance page of Box Office Mojo, a great datasource for box office performance data.

> Generally tabular data that is visible on the page will be put into 'tr' tags. With some of the code below I am exploring edge cases in the ouput of the page. When there is a special occasion for the weekend, in this case thanksgiving, there is a different format and structure that is displayed.

In [ ]:
# Request to website and download HTML contents
url='https://www.boxofficemojo.com/weekend/by-year/2024/'


#### Writing a function which scrapes the box office contents

In [131]:

def scrape_for_year(year):

    url = f'https://www.boxofficemojo.com/weekend/by-year/{year}/'
    response=""
    try:
        response = requests.get(url)
    except Exception as ex:
        print(f"Exception occurred {ex}")
        traceback.print_exc()
    content = response.text
    ## Reading webpage using beatifulsoup method available in bs4
    soup = BeautifulSoup(content)
    ## Find the tabular data that is visible on the page will be put into 'tr' tags
    rows = soup.findAll('tr')

    appended_data = []
    for row in rows:
        data_row = {}
        data_row['year'] = yr
        data = row.findAll('td')
        if len(data) == 0:
            continue
        if len(data[0].findAll('span')) > 0:
            #Data for special weekend
            data_row['occasion'] = data[0].findAll('span')[0].text
            data_row['date'] = data[0].findAll('a')[0].text
        else:
            data_row['occasion'] = "normal weekend"
            data_row['date'] = data[0].text
        data_row['top10_gross'] = data[1].text
        data_row['top10_wow_change'] = data[2].text
        data_row['overall_gross'] = data[3].text
        data_row['overall_wow_change'] = data[4].text
        data_row['num_releases'] = data[5].text
        data_row['top_release'] = data[6].text
        data_row['week_no'] = data[10].text
        appended_data.append(data_row)
    return appended_data

In [161]:
weekend_data = pd.DataFrame()
#Get last 10 years of box office collections
years = list(range(2013,2024,1))
years

[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

#### Creating Dataframe from the appended yearly data

In [162]:

for yr in years:
    result = scrape_for_year(yr)
    scrapped_df = pd.DataFrame(result, columns = ['date','occasion','year', 'top10_gross', 'top10_wow_change', 'overall_gross', 'overall_wow_change', 'num_releases', 'top_release', 'week_no'])
    weekend_data = pd.concat([weekend_data,scrapped_df],ignore_index=True)

In [163]:
weekend_data.shape

(677, 10)

In [134]:
weekend_data.head()

,date,occasion,year,top10_gross,top10_wow_change,overall_gross,overall_wow_change,num_releases,top_release,week_no
0,Dec 25-27,COVID-19 Pandemic,2020,"$23,780,363",+253.4%,"$24,226,397",+217.7%,34,Wonder Woman 1984,52
1,Dec 18-20,COVID-19 Pandemic,2020,"$6,729,235",+20.2%,"$7,625,545",+13.8%,41,Monster Hunter,51
2,Dec 11-13,COVID-19 Pandemic,2020,"$5,596,422",-24.7%,"$6,701,727",-20.2%,38,The Croods: A New Age,50
3,Dec 4-6,COVID-19 PandemicPost-Thanksgiving,2020,"$7,435,065",-43.8%,"$8,397,393",-40.7%,41,The Croods: A New Age,49
4,Nov 27-29,COVID-19 PandemicThanksgiving 3-Day,2020,"$13,230,347",+145.8%,"$14,152,936",+120.9%,41,The Croods: A New Age,48


In [35]:
weekend_data.columns

Index(['date', 'occasion', 'year', 'top10_gross', 'top10_wow_change',
       'overall_gross', 'overall_wow_change', 'num_releases', 'top_release',
       'week_no'],
      dtype='object')

#### Step 2: Removing null rows from the dataset

In [ ]:
## Calculating number of null rows present in the dataset


In [112]:
weekend_data.occasion.value_counts()

occasion
normal weekend                               177
COVID-19 Pandemic                             64
World Cup (Russia)                             5
MLK wknd                                       4
Indig. Peoples' Day wknd                       4
Labor Day wknd                                 4
New Year's long wknd                           3
Memorial Day wknd                              3
PyeongChang Olympics                           3
Easter wknd                                    3
Presidents' Day wknd                           3
July 4th long wknd                             3
Thanksgiving 5-Day                             3
Thanksgiving 4-Day                             3
Thanksgiving 3-Day                             3
Post-Thanksgiving                              3
Christmas long wknd                            3
World Cup (Qatar)                              2
COVID-19 PandemicMemorial Day wknd             2
COVID-19 PandemicEaster wknd                   2
COVID-19 Pa

In [164]:
scraped_movie_list=weekend_data['top_release'].unique()
len(scraped_movie_list)

353

#API

##### Step 1: Reading API data using "requests" library available in Python

Before invoking the API, you need to signup for your [TMDB](https://www.themoviedb.org/) account and get your api key to work with tmdb API.

#### 

In [1]:
api_key = '5a51d21e02c0baae3a96d6a5e7687635'

In [81]:
genre_url=f'https://api.themoviedb.org/3/genre/movie/list?api_key={api_key}'
genre_dict = dict()
try:
    response = requests.get(genre_url)
    if response.status_code == 200:
            genre_dict = { genre_json['id']:genre_json['name'] for genre_json in response.json()['genres'] } 
except Exception as e:
    print('Error downloading genres: ',e)
    traceback.print_exc()
    
#Lets add a unique key for movies without any genre
if 0 not in genre_dict.keys():
    genre_dict[0] = 'unknown'


To fetch the details we will use get method of request library and store retrieved information in a response object

In [165]:
%%time
# We will first create an empty dataframe to store all the movie detail
api_df = pd.DataFrame()
# Our for loop will iterate through each page, get json data convert it into dataframe and append it to original dataframe
for title in scraped_movie_list:
    url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={title}"
    response = requests.get(url)
    temporary_df = pd.DataFrame(response.json()['results'])
    api_df = pd.concat([api_df,temporary_df],ignore_index=True)

CPU times: total: 62.5 ms
Wall time: 29.8 s


Response object now has all our needed details but they are in the form of JSON. We can use json() to view these details

#### Now if you observe JSON carefully it has four keys

- page: shows page no. of current page
- results: contains list of dictionary with 20 movies
- total_pages: total pages present on site which contains the detail about movies
- total_results: total popular movies

#### We can convert this JSON into pandas dataframe
- Note: we are converting only 'results' dictionary into dataframe as we only need movie details

In [166]:
api_df.shape

(2998, 14)

In [43]:
api_df.columns

Index(['adult', 'backdrop_path', 'genre_ids', 'id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'title', 'video', 'vote_average', 'vote_count'],
      dtype='object')

In [167]:
api_df.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/npCPnwDyWfQltGfIZKN6WqeUXGI.jpg,"[14, 12, 28]",57158,en,The Hobbit: The Desolation of Smaug,"The Dwarves, Bilbo and Gandalf have successful...",93.446,/xQYiXsheRCDBA39DOrmaw1aSpbk.jpg,2013-12-11,The Hobbit: The Desolation of Smaug,False,7.574,13137.0
1,False,/u2bZhH3nTf0So0UIC1QxAqBvC07.jpg,"[16, 10751, 12, 14]",109445,en,Frozen,Young princess Anna of Arendelle dreams about ...,127.240,/mbPrrbt8bSLcHSBCHnRclPlMZPl.jpg,2013-11-20,Frozen,False,7.200,16504.0
2,False,/vD1yKObsRS2cvpmtuaCaMhr4zxe.jpg,[53],44363,en,Frozen,When three skiers find themselves stranded on ...,19.424,/2J3URUnDrIpNvh0uVqINQvr4HhW.jpg,2010-02-05,Frozen,False,5.996,1815.0
3,False,/sGjhRHNiQSkgVec18D3oX45hPmz.jpg,[53],26041,en,Frozen,It's two years since the mysterious disappeara...,3.312,/a6RlPQUerliQLkAieku5B8Loamk.jpg,2005-03-12,Frozen,False,5.700,18.0
4,False,None,"[18, 27]",170986,hi,Frozen,This is a touching and somber journey of Lasya...,1.903,/2GL9yZtrgbYKCeKBc3TF9gGfZpX.jpg,2007-07-21,Frozen,False,7.200,4.0


In [73]:
api_df['release_date'].value_counts().sort_index(ascending=False)

release_date
2026-10-01     1
2026-02-25     1
2025-12-03     1
2025-10-23     1
2025-07-30     1
              ..
1914-05-12     1
1913-09-26     1
1912-10-28     1
1912-10-01     1
              56
Name: count, Length: 867, dtype: int64

#Store all pickles for now

In [168]:
api_df.to_pickle("api_df_pickle")

In [169]:
weekend_data.to_pickle("web_scrapped_df_pickle")

In [48]:
pickle_api= pd.read_pickle("api_df_pickle")

In [4]:
web_scrape_pickle = pd.read_pickle("web_scrapped_df_pickle")

##Merge Datasource

In [172]:
csv_movies_with_budget_df.shape

(500, 13)

In [173]:
pickle_api.shape

(2998, 14)

In [174]:
web_scrape_pickle.shape

(677, 10)

In [175]:
csv_api_df = csv_movies_with_budget_df.merge(pickle_api,left_on='title', right_on="original_title")
csv_api_df.shape

(344, 27)

In [176]:
csv_api_df.head()

,rank,release_date_x,title_x,url,production_cost,domestic_gross,worldwide_gross,opening_weekend,mpaa,genre,...,original_language,original_title,overview,popularity,poster_path,release_date_y,title_y,video,vote_average,vote_count
0,1,2019-04-23,Avengers: Endgame,/movie/Avengers-Endgame-(2019)#tab=summary,400000000,858373000,2797800564,357115007.0,PG-13,Action,...,en,Avengers: Endgame,After the devastating events of Avengers: Infi...,107.963,/or06FN3Dka5tukK1e9sl16pB3iy.jpg,2019-04-24,Avengers: Endgame,False,8.247,25669.0
1,1,2019-04-23,Avengers: Endgame,/movie/Avengers-Endgame-(2019)#tab=summary,400000000,858373000,2797800564,357115007.0,PG-13,Action,...,en,Avengers: Endgame,After the devastating events of Avengers: Infi...,107.963,/or06FN3Dka5tukK1e9sl16pB3iy.jpg,2019-04-24,Avengers: Endgame,False,8.247,25670.0
2,3,2015-04-22,Avengers: Age of Ultron,/movie/Avengers-Age-of-Ultron#tab=summary,365000000,459005868,1395316979,191271109.0,PG-13,Action,...,en,Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,69.688,/4ssDuvEDkSArWEdyBl2X5EHvYKU.jpg,2015-04-22,Avengers: Age of Ultron,False,7.274,22971.0
3,3,2015-04-22,Avengers: Age of Ultron,/movie/Avengers-Age-of-Ultron#tab=summary,365000000,459005868,1395316979,191271109.0,PG-13,Action,...,en,Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,69.688,/4ssDuvEDkSArWEdyBl2X5EHvYKU.jpg,2015-04-22,Avengers: Age of Ultron,False,7.274,22971.0
4,5,2018-04-25,Avengers: Infinity War,/movie/Avengers-Infinity-War#tab=summary,300000000,678815482,2048359754,257698183.0,PG-13,Action,...,en,Avengers: Infinity War,As the Avengers and their allies have continue...,223.221,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,2018-04-25,Avengers: Infinity War,False,8.200,29799.0


In [177]:
csv_web_scrape_df = csv_movies_with_budget_df.merge(web_scrape_pickle,left_on='title', right_on="top_release")
csv_web_scrape_df.shape

(293, 23)

In [178]:
scrape_csv_api_df = web_scrape_pickle.merge(csv_api_df,left_on='top_release', right_on="original_title")
scrape_csv_api_df.shape

(653, 37)

In [179]:
scrape_csv_api_df.head()

,date,occasion,year_x,top10_gross,top10_wow_change,overall_gross,overall_wow_change,num_releases,top_release,week_no,...,original_language,original_title,overview,popularity,poster_path,release_date_y,title_y,video,vote_average,vote_count
0,Dec 27-29,normal weekend,2013,"$167,837,974",+24.5%,"$197,177,755",+37.3%,81,The Hobbit: The Desolation of Smaug,52,...,en,The Hobbit: The Desolation of Smaug,"The Dwarves, Bilbo and Gandalf have successful...",93.446,/xQYiXsheRCDBA39DOrmaw1aSpbk.jpg,2013-12-11,The Hobbit: The Desolation of Smaug,False,7.574,13137.0
1,Dec 20-22,normal weekend,2013,"$134,837,792",-2.6%,"$143,571,365",-2.8%,80,The Hobbit: The Desolation of Smaug,51,...,en,The Hobbit: The Desolation of Smaug,"The Dwarves, Bilbo and Gandalf have successful...",93.446,/xQYiXsheRCDBA39DOrmaw1aSpbk.jpg,2013-12-11,The Hobbit: The Desolation of Smaug,False,7.574,13137.0
2,Dec 13-15,normal weekend,2013,"$138,369,003",+64.8%,"$147,702,714",+56.2%,94,The Hobbit: The Desolation of Smaug,50,...,en,The Hobbit: The Desolation of Smaug,"The Dwarves, Bilbo and Gandalf have successful...",93.446,/xQYiXsheRCDBA39DOrmaw1aSpbk.jpg,2013-12-11,The Hobbit: The Desolation of Smaug,False,7.574,13137.0
3,Dec 6-8,Post-Thanksgiving,2013,"$83,941,456",-55.7%,"$94,535,353",-54.6%,96,Frozen,49,...,en,Frozen,Young princess Anna of Arendelle dreams about ...,127.240,/mbPrrbt8bSLcHSBCHnRclPlMZPl.jpg,2013-11-20,Frozen,False,7.200,16504.0
4,Dec 6-8,Post-Thanksgiving,2013,"$83,941,456",-55.7%,"$94,535,353",-54.6%,96,Frozen,49,...,en,Frozen,When three skiers find themselves stranded on ...,19.424,/2J3URUnDrIpNvh0uVqINQvr4HhW.jpg,2010-02-05,Frozen,False,5.996,1815.0


##Milestone 2

#### *Missing values in the dataset*

In [181]:
csv_movies_with_budget_df.isnull().sum()

rank                0
release_date        1
title               0
url                 0
production_cost     0
domestic_gross      0
worldwide_gross     0
opening_weekend    21
mpaa                8
genre               5
theaters           21
runtime            13
year                1
dtype: int64

All titles are available
Although the release date column for a value is null, the year column is not null
Some genre is null

In [211]:
csv_movies_with_budget_df.isnull().any(axis=1).sum()

0

In [210]:
csv_movies_with_budget_df[csv_movies_with_budget_df['genre'].isnull()].year.value_counts()

Series([], Name: count, dtype: int64)

Only 1 movie belonging to 2015 has an empty genre, the other years are not relevant for my analysis.

There is a small number of null values in the dataset for the columns. Since some of them are not useful for my analysis, i will drop them during the data cleaning section.

Duplicates in the dataset

In [193]:
sum(csv_movies_with_budget_df.duplicated())

0

#### *Number of unique values in the dataset*

In [209]:
# Returns the number of unique values in each column
csv_movies_with_budget_df.nunique()

rank               474
release_date       454
title              472
production_cost     89
domestic_gross     474
worldwide_gross    474
opening_weekend    472
genre               10
theaters           404
runtime             94
year                30
dtype: int64

## Data Cleaning (Handling missing values and duplicate data: Removing unused information)

In this section, i will try to handle the problem of missing values, duplicated data  and format:

<ul>
<li>Remove the unused columns that are not useful in my analysis; </li>
<li>Remove the movies  having empty genres;</li>
<li>Ensure there is no white spaces on the primay key, the movie title </li>
<li>Remove duplicate rows</li>
<li>Changing format of release date into datetime format and budget_adj/revenue_adj format from float to int.</li>
</ul>

In [199]:
# drop unused columns
csv_movies_with_budget_df.drop(['url','mpaa'], axis=1, inplace=True)
req_cols = ['title','genre']
csv_movies_with_budget_df[req_cols] = csv_movies_with_budget_df[req_cols].apply(lambda col:col.str.strip())


KeyError: "['url', 'mpaa'] not found in axis"

In [201]:
# drop duplicates
csv_movies_with_budget_df.drop_duplicates(inplace=True)
# confirm correction by rechecking for duplicates in the data
sum(csv_movies_with_budget_df.duplicated())

0

In [207]:
# drop rows with any null values 
csv_movies_with_budget_df.dropna(subset=['genre'],inplace=True)
csv_movies_with_budget_df.dropna(inplace=True)
# checks if any of columns in the data have null values - should print False
csv_movies_with_budget_df.isnull().sum().any()

False

In [204]:
# Convert 'release_date' type from str to datetime 
csv_movies_with_budget_df['release_date']=pd.to_datetime(csv_movies_with_budget_df['release_date'])
csv_movies_with_budget_df['release_date'].head()

0   2019-04-23
1   2011-05-20
2   2015-04-22
3   2015-12-16
4   2018-04-25
Name: release_date, dtype: datetime64[ns]

In [208]:
# Final number of movies 
rows, col = csv_movies_with_budget_df.shape
print('After cleaning, we now have only {} entries of movies.'.format(rows))

After cleaning, we now have only 474 entries of movies.


In [212]:
csv_movies_with_budget_df.head()

,rank,release_date,title,production_cost,domestic_gross,worldwide_gross,opening_weekend,genre,theaters,runtime,year
0,1,2019-04-23,Avengers: Endgame,400000000,858373000,2797800564,357115007.0,Action,4662.0,181.0,2019.0
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,379000000,241071802,1045713802,90151958.0,Adventure,4164.0,136.0,2011.0
2,3,2015-04-22,Avengers: Age of Ultron,365000000,459005868,1395316979,191271109.0,Action,4276.0,141.0,2015.0
3,4,2015-12-16,Star Wars Ep. VII: The Force Awakens,306000000,936662225,2064615817,247966675.0,Adventure,4134.0,136.0,2015.0
4,5,2018-04-25,Avengers: Infinity War,300000000,678815482,2048359754,257698183.0,Action,4474.0,156.0,2018.0


#Milestone-3

#### Step 1: Reading website data using beautifulSoup library available in Python
#### Step 2: Removing null rows from the dataset
#### Step 2: Removing null rows from the dataset
#### Step 2: Removing null rows from the dataset
#### Step 2: Removing null rows from the dataset
#### Ethical implications of data wrangling specific to your datasource and the steps you completed answering the following questions:
### What changes were made to the data?
### Are there any legal or regulatory guidelines for your data or project topic?
### What risks could be created based on the transformations done?
### Did you make any assumptions in cleaning/transforming the data?
### How was your data sourced / verified for credibility?
### Was your data acquired in an ethical way?
### How would you mitigate any of the ethical implications you have identified?

In [14]:
web_scrape_pickle.head()

,date,occasion,year,top10_gross,top10_wow_change,overall_gross,overall_wow_change,num_releases,movie_title,week_no
0,Dec 27-29,normal weekend,2013,"$167,837,974",+24.5%,"$197,177,755",+37.3%,81,The Hobbit: The Desolation of Smaug,52
1,Dec 20-22,normal weekend,2013,"$134,837,792",-2.6%,"$143,571,365",-2.8%,80,The Hobbit: The Desolation of Smaug,51
2,Dec 13-15,normal weekend,2013,"$138,369,003",+64.8%,"$147,702,714",+56.2%,94,The Hobbit: The Desolation of Smaug,50
3,Dec 6-8,Post-Thanksgiving,2013,"$83,941,456",-55.7%,"$94,535,353",-54.6%,96,Frozen,49
4,Nov 29-Dec 1,Thanksgiving 3-Day,2013,"$189,483,142",-11.7%,"$208,125,032",-8.1%,104,The Hunger Games: Catching Fire,48


In [22]:
# Remove dollar signs and comma convert to integer
web_scrape_pickle['top10_gross'] = web_scrape_pickle['top10_gross'].replace('[$,]', '', regex=True).astype(int)
web_scrape_pickle['overall_gross'] = web_scrape_pickle['overall_gross'].replace('[$,]', '', regex=True).astype(int)
web_scrape_pickle.head()


,date,occasion,year,top10_gross,top10_wow_change,overall_gross,overall_wow_change,num_releases,movie_title,week_no
0,Dec 27-29,normal weekend,2013,167837974,+24.5%,197177755,+37.3%,81,The Hobbit: The Desolation of Smaug,52
1,Dec 20-22,normal weekend,2013,134837792,-2.6%,143571365,-2.8%,80,The Hobbit: The Desolation of Smaug,51
2,Dec 13-15,normal weekend,2013,138369003,+64.8%,147702714,+56.2%,94,The Hobbit: The Desolation of Smaug,50
3,Dec 6-8,Post-Thanksgiving,2013,83941456,-55.7%,94535353,-54.6%,96,Frozen,49
4,Nov 29-Dec 1,Thanksgiving 3-Day,2013,189483142,-11.7%,208125032,-8.1%,104,The Hunger Games: Catching Fire,48


In [47]:
# Create the column in millions
web_scrape_pickle['top_10_gross_in_millions'] = web_scrape_pickle['top10_gross'] / 1000000
web_scrape_pickle['top_10_gross_in_millions'] = web_scrape_pickle['top_10_gross_in_millions'].apply(lambda x: f"{x:.2f}")

web_scrape_pickle['overall_gross_in_millions'] = web_scrape_pickle['overall_gross'] / 1000000
web_scrape_pickle['overall_gross_in_millions'] = web_scrape_pickle['overall_gross_in_millions'].apply(lambda x: f"{x:.2f}")

web_scrape_pickle.head()

,movie_title,year,date,occasion,top_10_gross_in_millions,overall_gross_in_millions,top10_wow_change,overall_wow_change,top10_gross,overall_gross,num_releases,week_no
0,The Hobbit: The Desolation of Smaug,2013,Dec 27-29,normal weekend,167.84,197.18,+24.5%,+37.3%,167837974,197177755,81,52
1,The Hobbit: The Desolation of Smaug,2013,Dec 20-22,normal weekend,134.84,143.57,-2.6%,-2.8%,134837792,143571365,80,51
2,The Hobbit: The Desolation of Smaug,2013,Dec 13-15,normal weekend,138.37,147.70,+64.8%,+56.2%,138369003,147702714,94,50
3,Frozen,2013,Dec 6-8,Post-Thanksgiving,83.94,94.54,-55.7%,-54.6%,83941456,94535353,96,49
4,The Hunger Games: Catching Fire,2013,Nov 29-Dec 1,Thanksgiving 3-Day,189.48,208.13,-11.7%,-8.1%,189483142,208125032,104,48


#### Step 4: Convert gross columns to float

In [50]:
web_scrape_pickle['top_10_gross_in_millions'] = web_scrape_pickle['top_10_gross_in_millions'].astype(float)
web_scrape_pickle['overall_gross_in_millions'] = web_scrape_pickle['overall_gross_in_millions'].astype(float)


#### Step 5: Replace Headers with more meaningful names

In [43]:

web_scrape_pickle.rename(columns={'top_release': 'movie_title'}, inplace=True)

In [51]:
web_scrape_pickle.head()

,movie_title,year,date,occasion,top_10_gross_in_millions,overall_gross_in_millions,top10_wow_change,overall_wow_change,top10_gross,overall_gross,num_releases,week_no
0,The Hobbit: The Desolation of Smaug,2013,Dec 27-29,normal weekend,167.84,197.18,+24.5%,+37.3%,167837974,197177755,81,52
1,The Hobbit: The Desolation of Smaug,2013,Dec 20-22,normal weekend,134.84,143.57,-2.6%,-2.8%,134837792,143571365,80,51
2,The Hobbit: The Desolation of Smaug,2013,Dec 13-15,normal weekend,138.37,147.70,+64.8%,+56.2%,138369003,147702714,94,50
3,Frozen,2013,Dec 6-8,Post-Thanksgiving,83.94,94.54,-55.7%,-54.6%,83941456,94535353,96,49
4,The Hunger Games: Catching Fire,2013,Nov 29-Dec 1,Thanksgiving 3-Day,189.48,208.13,-11.7%,-8.1%,189483142,208125032,104,48


#### Step 6: Replace arrange Headers

In [48]:
new_col_order=['movie_title', 'year' ,'date', 'occasion', 'top_10_gross_in_millions', 'overall_gross_in_millions', 'top10_wow_change','overall_wow_change','top10_gross', 'overall_gross',  'num_releases', 
               'week_no']

for i,col in enumerate(new_col_order):
    tmp = web_scrape_pickle[col]
    web_scrape_pickle.drop(labels=[col],axis=1,inplace=True)
    web_scrape_pickle.insert(i,col,tmp) 

In [45]:
web_scrape_pickle.head()

,movie_title,year,date,occasion,top_10_gross_in_millions,overall_gross_in_millions,top10_wow_change,overall_wow_change,top10_gross,overall_gross,num_releases,week_no
0,The Hobbit: The Desolation of Smaug,2013,Dec 27-29,normal weekend,167.84,197.18,+24.5%,+37.3%,167837974,197177755,81,52
1,The Hobbit: The Desolation of Smaug,2013,Dec 20-22,normal weekend,134.84,143.57,-2.6%,-2.8%,134837792,143571365,80,51
2,The Hobbit: The Desolation of Smaug,2013,Dec 13-15,normal weekend,138.37,147.70,+64.8%,+56.2%,138369003,147702714,94,50
3,Frozen,2013,Dec 6-8,Post-Thanksgiving,83.94,94.54,-55.7%,-54.6%,83941456,94535353,96,49
4,The Hunger Games: Catching Fire,2013,Nov 29-Dec 1,Thanksgiving 3-Day,189.48,208.13,-11.7%,-8.1%,189483142,208125032,104,48


In [49]:
web_scrape_pickle.groupby(['movie_title','year'])['overall_gross_in_millions'].sum().sort_values(ascending=False)

movie_title                              year
Black Panther                            2018    1250.21
Star Wars: Episode VIII - The Last Jedi  2017    1163.35
Rogue One: A Star Wars Story             2016    1101.12
The Hunger Games: Mockingjay - Part 2    2015    1004.10
Ralph Breaks the Internet                2018     994.79
                                                  ...   
Phoenix, OregonRe-release                2020       0.01
Resistance                               2020       0.01
True History of the Kelly Gang           2020       0.01
Strike                                   2020       0.00
Swallow                                  2020       0.00
Name: overall_gross_in_millions, Length: 362, dtype: float64

Ethical implications


Take the root of the url, in this case https://www.boxofficemojo.com and add '/robots.txt' to the end. This will come up with a page that shows what type of web scraping is allowed or disallowed. Thankfully Box Office Mojo allows all

#Milestone -4

In [118]:
pickle_api = pd.read_pickle("api_df_pickle")

In [119]:
#pandas replace empty square brackets with [0], here genre[0] is unknown which we created earlier
pickle_api['genre_ids'] = pickle_api['genre_ids'].apply(lambda  x : [0] if not x else x)



In [120]:
#First drop na columns
pickle_api = pickle_api.dropna(subset=['genre_ids'])
#then map the genres
pickle_api['genre_ids'] = pickle_api['genre_ids'].apply(lambda x: [genre_dict[i] for i in x ])



In [121]:
pickle_api.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/npCPnwDyWfQltGfIZKN6WqeUXGI.jpg,"[Fantasy, Adventure, Action]",57158,en,The Hobbit: The Desolation of Smaug,"The Dwarves, Bilbo and Gandalf have successful...",93.446,/xQYiXsheRCDBA39DOrmaw1aSpbk.jpg,2013-12-11,The Hobbit: The Desolation of Smaug,False,7.574,13137.0
1,False,/u2bZhH3nTf0So0UIC1QxAqBvC07.jpg,"[Animation, Family, Adventure, Fantasy]",109445,en,Frozen,Young princess Anna of Arendelle dreams about ...,127.240,/mbPrrbt8bSLcHSBCHnRclPlMZPl.jpg,2013-11-20,Frozen,False,7.200,16504.0
2,False,/vD1yKObsRS2cvpmtuaCaMhr4zxe.jpg,[Thriller],44363,en,Frozen,When three skiers find themselves stranded on ...,19.424,/2J3URUnDrIpNvh0uVqINQvr4HhW.jpg,2010-02-05,Frozen,False,5.996,1815.0
3,False,/sGjhRHNiQSkgVec18D3oX45hPmz.jpg,[Thriller],26041,en,Frozen,It's two years since the mysterious disappeara...,3.312,/a6RlPQUerliQLkAieku5B8Loamk.jpg,2005-03-12,Frozen,False,5.700,18.0
4,False,None,"[Drama, Horror]",170986,hi,Frozen,This is a touching and somber journey of Lasya...,1.903,/2GL9yZtrgbYKCeKBc3TF9gGfZpX.jpg,2007-07-21,Frozen,False,7.200,4.0


In [122]:
#removing brackets from list type inside pandas cell
pickle_api['genre_ids'] = pickle_api['genre_ids'].str.join(',')


In [124]:
pickle_api.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/npCPnwDyWfQltGfIZKN6WqeUXGI.jpg,"Fantasy,Adventure,Action",57158,en,The Hobbit: The Desolation of Smaug,"The Dwarves, Bilbo and Gandalf have successful...",93.446,/xQYiXsheRCDBA39DOrmaw1aSpbk.jpg,2013-12-11,The Hobbit: The Desolation of Smaug,False,7.574,13137.0
1,False,/u2bZhH3nTf0So0UIC1QxAqBvC07.jpg,"Animation,Family,Adventure,Fantasy",109445,en,Frozen,Young princess Anna of Arendelle dreams about ...,127.240,/mbPrrbt8bSLcHSBCHnRclPlMZPl.jpg,2013-11-20,Frozen,False,7.200,16504.0
2,False,/vD1yKObsRS2cvpmtuaCaMhr4zxe.jpg,Thriller,44363,en,Frozen,When three skiers find themselves stranded on ...,19.424,/2J3URUnDrIpNvh0uVqINQvr4HhW.jpg,2010-02-05,Frozen,False,5.996,1815.0
3,False,/sGjhRHNiQSkgVec18D3oX45hPmz.jpg,Thriller,26041,en,Frozen,It's two years since the mysterious disappeara...,3.312,/a6RlPQUerliQLkAieku5B8Loamk.jpg,2005-03-12,Frozen,False,5.700,18.0
4,False,None,"Drama,Horror",170986,hi,Frozen,This is a touching and somber journey of Lasya...,1.903,/2GL9yZtrgbYKCeKBc3TF9gGfZpX.jpg,2007-07-21,Frozen,False,7.200,4.0


In [123]:
pickle_api['genre_ids'].value_counts()

genre_ids
unknown                                              328
Documentary                                          260
Drama                                                188
Comedy                                               129
Horror                                               124
                                                    ... 
Thriller,Mystery,Drama,Crime                           1
Documentary,History,Crime                              1
Crime,Thriller,Action                                  1
TV Movie,Action,Adventure,Fantasy,Science Fiction      1
Horror,Mystery,Thriller,Comedy,Romance                 1
Name: count, Length: 766, dtype: int64

#Milestone -5

In [47]:
# importing sqlite3 library
import sqlalchemy
from sqlalchemy import create_engine , select, MetaData, Table, and_
import pandas as pd
import sqlite3

In [48]:
engine = sqlalchemy.create_engine('sqlite:///movie_world.db', echo=False)

In [50]:
api_df.to_sql('2021_22_api_results', con=engine, if_exists='append')

ProgrammingError: (sqlite3.ProgrammingError) Error binding parameter 4: type 'list' is not supported
[SQL: INSERT INTO "2021_22_api_results" ("index", adult, backdrop_path, genre_ids, id, original_language, original_title, overview, popularity, poster_path, release_date, title, video, vote_average, vote_count) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: [(0, 0, '/2QR0OpYzPFG84EswxENqcPuHmP.jpg', [10752], 1041317, 'en', 'Battle for the Western Front', 'Inspired by Canuck WWI flying ace Billy Bishop, two young soldiers crash the front lines in August 1918. With the Allies detained, their platoon must desperately hold out alongside legendary Anishinaabe sniper Francis "Peggy" Pegahmagabow.', 291.934, '/ooHEL3XyLYu3hus5cMnpjeH4e7A.jpg', '2022-10-06', 'Battle for the Western Front', 0, 5.65, 10), (1, 0, '/vIgyYkXkg6NC2whRbYjBD7eb3Er.jpg', [878, 28, 12], 580489, 'en', 'Venom: Let There Be Carnage', 'After finding a host body in investigative reporter Eddie Brock, the a